In [1]:
from os import listdir
from pandas import read_csv
import pandas as pd
import glob

In [1]:
# Step 1: Download the zip file
import urllib.request

url = "https://archive.ics.uci.edu/ml/datasets/human+activity+recognition+using+smartphones"
filename = "Activity_Recognition_from_Single_Chest-Mounted_Accelerometer.zip"

urllib.request.urlretrieve(url, filename)

# Step 2: Extract the zip file
import zipfile

with zipfile.ZipFile(filename, 'r') as zip_ref:
    zip_ref.extractall()

csv_files = glob.glob("Activity Recognition from Single Chest-Mounted Accelerometer/1.csv".format('csv'))
csv_files.append(("Activity Recognition from Single Chest-Mounted Accelerometer/2.csv"))
csv_files.append(("Activity Recognition from Single Chest-Mounted Accelerometer/3.csv"))
csv_files.append(("Activity Recognition from Single Chest-Mounted Accelerometer/4.csv"))
csv_files.append(("Activity Recognition from Single Chest-Mounted Accelerometer/5.csv"))
csv_files.append(("Activity Recognition from Single Chest-Mounted Accelerometer/6.csv"))
csv_files.append(("Activity Recognition from Single Chest-Mounted Accelerometer/7.csv"))
csv_files.append(("Activity Recognition from Single Chest-Mounted Accelerometer/8.csv"))
csv_files.append(("Activity Recognition from Single Chest-Mounted Accelerometer/9.csv"))
csv_files.append(("Activity Recognition from Single Chest-Mounted Accelerometer/10.csv"))
csv_files.append(("Activity Recognition from Single Chest-Mounted Accelerometer/11.csv"))
csv_files.append(("Activity Recognition from Single Chest-Mounted Accelerometer/12.csv"))
csv_files.append(("Activity Recognition from Single Chest-Mounted Accelerometer/13.csv"))
csv_files.append(("Activity Recognition from Single Chest-Mounted Accelerometer/14.csv"))
csv_files.append(("Activity Recognition from Single Chest-Mounted Accelerometer/15.csv"))

csv_files

# Step 3: Load the dataset
df_concat = pd.concat([pd.read_csv(f) for f in csv_files ], ignore_index=True)
df_concat

# View the data
print(df_concat.head())

BadZipFile: File is not a zip file